# Background

Here I provide code for concatenating the .wav files from each of the birdcall directories into a single larger .wav file.

I decided to do so after reading the winning solution in the BirdCLEF 2019 competition. [Bird Species Identification in Soundscapes](http://ceur-ws.org/Vol-2380/paper_86.pdf)

In his write-up after the competition, Mario Lasseck describes a data augmentation technique in which he uses a single large file composed of background noise from various soundscapes. Similarly, I decided to use a similar scheme in which I concat a large wav file and pick 5 sec segments out rather than dealing with more complicated logic to account for wav files that are less than 5 seconds or (not evenly divisible into 5 second intervals).

It took me a few hours to figure out how to call ffmpeg from within Python, so I provide the code here in case it's of assistance to someone. Additionally, I've provided the dataset that was produced from this code. More details on the dataset at the links below:

# Dataset
[A to B](https://www.kaggle.com/smodad/birdcall-no-background-concat-a-b)

[C to F](https://www.kaggle.com/smodad/birdcall-no-background-concat-cf)

[G to M](https://www.kaggle.com/smodad/birdcall-no-background-concat-gm)

[N to R](https://www.kaggle.com/smodad/birdcall-no-background-concat-nr)

[S to Y](https://www.kaggle.com/smodad/birdcall-no-background-concat-sy)

[Background noise](https://www.kaggle.com/smodad/birdcall-background-concat)



In [ ]:
import os
from pathlib import Path
import subprocess #module for launching process from within Python

# Creating the .txt file for ffmpeg to process

In order to automate the concat process, ffmpeg requires a .txt file with a path to each .wav file you wish to merge together. In order to create this .txt file, I create a list containing the name of each directory. Because the Cornell Birdcall Identification dataset is already sub-divided by eBirdcode (i.e., species), this makes it easy to create .wav files with the appropriate eBirdcode so that the data is more organized.

Each .txt file created contains a path to each .wav file in the species folder so that ffmpeg will concatenate  every .wav file in the folder to a single large file that contains all of the sounds for that species. I did this because it makes the data augmentation easier as some of the clips are less than 5 seconds but we need to pull 5 second clips for the labeling as part of the competition.

Below I'm using absolute pathnames because ffmpeg was giving me trouble with relative paths. You may also notice the "file" before the pathname. This is because ffmpeg requires the following format for .txt files it uses for concatenation:
    
    file 'file1.wav'
    file 'file2.wav'
    ...

Note: The ' ' are stated as not being necessary by ffmpeg but it's considered best practice to include them as some directories may have spaces.


In [ ]:
# os.listdir(directory) makes a list of all subdirectories in 'directory' (specified as the main argument)
# Here, I placed all of the birdcall folders (aldfly, ... , yetvir) in the directory 'call'
dir_list = os.listdir('call')


for directory in dir_list:
    filename_list = os.listdir('./call/' + directory) # Make a list of filenames for each sub-directory in 'call'
    file_object = open(directory + '.txt', 'w') # Create a .txt file named according to the species
    with open('./' + directory + '.txt', 'w') as filehandle:
        for filename in filename_list:
            # This creates a .txt file where each filename is listed per line for ffmpeg to process
            filehandle.write("file 'D:\\Kaggle\\birdcall\\call\\"  + directory + "\\" + "%s" % filename + "'\n")
    file_object.close()

# Calling ffmpeg using subprocess.run()

Here I create a string first and then pass that string to subprocess.run() which sends the ffmpeg command to the system. When *shell=True*, subprocess.run() will use the first argument supplied as a string to pass directly to the OS. *check=True* will raise an exception if the process raises an exception on exit. Fortunately, subprocess.run() will wait for the process to end before creating a new process, so you can run it in the background without fear of memory problems.

The command line string is of the form:

    ffmpeg -f concat -safe 0 -i path_to/input_file.txt -c copy path_to/concat_file.wav

The run-time on my local machine (8 cores) was around 30 minutes using the CPU.

Note: Not all audio file formats work properly with ffmpeg. More information on using ffmpeg to concat audio files can be found here: https://trac.ffmpeg.org/wiki/Concatenate

In [ ]:
for directory in dir_list:
    cmdargs = "ffmpeg -f concat -safe 0 -i " + directory + ".txt -c copy ./call/" + directory + "/" + directory + ".wav"
    # subprocess.run() will allow the process to end before creating a new one
    subprocess.run(cmdargs, check=True, shell=True)